<a href="https://colab.research.google.com/github/pedro-pauletti/nlp-with-transformers/blob/main/Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Requirements

In [ ]:
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.2 MB/s eta 0:00:00


### Extract The Last Hidden State Tensor

In [ ]:
text = "hello world what a time to be alive!"

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
tokens = tokenizer.encode_plus(text, max_length=128,
                               truncation=True, padding='max_length',
                               return_tensors='pt')

In [ ]:
outputs= model(**tokens)

In [ ]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 3.0681e-01, -7.8805e-02,  1.7431e+00,  ..., -2.5349e-02,
          -1.1080e-01,  4.8311e-02],
         [ 7.1302e-01,  1.0437e-01,  1.8346e+00,  ...,  1.1343e-01,
          -7.5564e-02,  1.2668e-01],
         [ 8.1722e-01,  1.1321e-01,  1.5408e+00,  ..., -3.8067e-01,
           8.7477e-02, -1.9020e-01],
         ...,
         [ 5.4669e-01,  1.7181e-01,  1.1392e+00,  ...,  3.8548e-02,
          -1.5396e-01,  2.3015e-01],
         [ 3.4457e-01,  1.3151e-01,  1.1324e+00,  ..., -1.4210e-03,
          -1.7517e-01,  1.5220e-01],
         [ 3.2320e-01,  3.3353e-03,  1.1888e+00,  ...,  1.6736e-02,
          -2.0864e-01,  8.9316e-02]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-7.1334e-01, -2.7907e-01,  7.7808e-01, -1.9139e-01, -1.4626e-01,
         -2.2970e-01,  4.3867e-01, -1.3565e-01,  4.1929e-01, -8.8569e-01,
          3.7006e-01, -3.7974e-01,  7.3328e-01, -2.6172e-01,  8.7081e-01,
         -3.7279e-

In [ ]:
embeddings = outputs.last_hidden_state
embeddings

tensor([[[ 3.0681e-01, -7.8805e-02,  1.7431e+00,  ..., -2.5349e-02,
          -1.1080e-01,  4.8311e-02],
         [ 7.1302e-01,  1.0437e-01,  1.8346e+00,  ...,  1.1343e-01,
          -7.5564e-02,  1.2668e-01],
         [ 8.1722e-01,  1.1321e-01,  1.5408e+00,  ..., -3.8067e-01,
           8.7477e-02, -1.9020e-01],
         ...,
         [ 5.4669e-01,  1.7181e-01,  1.1392e+00,  ...,  3.8548e-02,
          -1.5396e-01,  2.3015e-01],
         [ 3.4457e-01,  1.3151e-01,  1.1324e+00,  ..., -1.4210e-03,
          -1.7517e-01,  1.5220e-01],
         [ 3.2320e-01,  3.3353e-03,  1.1888e+00,  ...,  1.6736e-02,
          -2.0864e-01,  8.9316e-02]]], grad_fn=<NativeLayerNormBackward0>)

In [ ]:
embeddings.shape

torch.Size([1, 128, 768])

### Sentence Vectors With Mean Pooling

In [ ]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([1, 128])

In [ ]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.shape).float()

In [ ]:
masked_embeddings = embeddings * mask
masked_embeddings

tensor([[[ 0.3068, -0.0788,  1.7431,  ..., -0.0253, -0.1108,  0.0483],
         [ 0.7130,  0.1044,  1.8346,  ...,  0.1134, -0.0756,  0.1267],
         [ 0.8172,  0.1132,  1.5408,  ..., -0.3807,  0.0875, -0.1902],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000]]],
       grad_fn=<MulBackward0>)

In [ ]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([1, 768])

In [ ]:
counts = torch.clamp(mask.sum(1), min=1e-9)
counts.shape

torch.Size([1, 768])

In [ ]:
mean_pooled = summed/counts
mean_pooled.shape

torch.Size([1, 768])

### Using Cosine Similarity

In [ ]:
sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "Standing on one's head at job interviews forms a lasting impression.",
    "It took him a month to finish the meal.",
    "He found a leprechaun in his walnut shell."
]


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
  new_tokens = tokenizer.encode_plus(text, max_length=128,
                               truncation=True, padding='max_length',
                               return_tensors='pt')
  tokens['input_ids'].append(new_tokens['input_ids'][0])
  tokens['attention_mask'].append(new_tokens['attention_mask'][0])

tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [ ]:
tokens['input_ids'].shape

torch.Size([6, 128])

In [ ]:
outputs = model(**tokens)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
embeddings = outputs.last_hidden_state

In [ ]:
embeddings.shape

torch.Size([6, 128, 768])

In [ ]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([6, 128])

In [ ]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.shape).float()

In [ ]:
masked_embeddings = embeddings * mask
masked_embeddings

tensor([[[ 0.3068, -0.0788,  1.7431,  ..., -0.0253, -0.1108,  0.0483],
         [ 0.7130,  0.1044,  1.8346,  ...,  0.1134, -0.0756,  0.1267],
         [ 0.8172,  0.1132,  1.5408,  ..., -0.3807,  0.0875, -0.1902],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000]],

        [[ 0.3068, -0.0788,  1.7431,  ..., -0.0253, -0.1108,  0.0483],
         [ 0.7130,  0.1044,  1.8346,  ...,  0.1134, -0.0756,  0.1267],
         [ 0.8172,  0.1132,  1.5408,  ..., -0.3807,  0.0875, -0.1902],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000]],

        [[ 0.3068, -0.0788,  1.7431,  ..., -0.0253, -0.1108,  0.0483],
         [ 0.7130,  0.1044,  1.8346,  ...,  0

In [ ]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([6, 768])

In [ ]:
counts = torch.clamp(mask.sum(1), min=1e-9)
counts.shape

torch.Size([6, 768])

In [ ]:
mean_pooled = summed/counts
mean_pooled.shape

torch.Size([6, 768])

In [ ]:
mean_pooled

tensor([[ 0.6437, -0.1671,  1.6938,  ...,  0.0823, -0.1704, -0.0838],
        [ 0.6437, -0.1671,  1.6938,  ...,  0.0823, -0.1704, -0.0838],
        [ 0.6437, -0.1671,  1.6938,  ...,  0.0823, -0.1704, -0.0838],
        [ 0.6437, -0.1671,  1.6938,  ...,  0.0823, -0.1704, -0.0838],
        [ 0.6437, -0.1671,  1.6938,  ...,  0.0823, -0.1704, -0.0838],
        [ 0.6437, -0.1671,  1.6938,  ...,  0.0823, -0.1704, -0.0838]],
       grad_fn=<DivBackward0>)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
mean_pooled = mean_pooled.detach().numpy()

cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[1.       , 1.       , 1.       , 1.       , 1.0000001]],
      dtype=float32)

### Similarity with Sentence-Transformers

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c5db813f2a43388bb940b864138540b545e037acd456cbfb7f26d875c8ebe1ad
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "Standing on one's head at job interviews forms a lasting impression.",
    "It took him a month to finish the meal.",
    "He found a leprechaun in his walnut shell."
]


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentence_embeddings = model.encode(sentences)

In [ ]:
sentence_embeddings.shape

(6, 768)

In [ ]:
sentence_embeddings

array([[ 0.07446145,  0.86369675,  0.17946443, ...,  0.7734398 ,
         1.7247484 , -0.18027496],
       [-0.37146327,  0.9729014 ,  1.0839936 , ..., -0.25521314,
        -0.275937  ,  0.03575847],
       [-0.5029822 ,  0.7949859 , -0.12402567, ...,  0.14406323,
         0.9703749 , -0.17911568],
       [-0.01324296,  0.977286  ,  1.4515941 , ..., -0.8461652 ,
        -1.4004321 , -0.41184363],
       [-0.20192628,  0.05970358,  0.8602744 , ..., -0.01000803,
         0.8430624 , -0.0840772 ],
       [-0.21311924,  1.0174936 , -0.883276  , ...,  0.7371039 ,
         0.19469169, -0.3011128 ]], dtype=float32)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)

array([[0.33088923, 0.7219259 , 0.17475507, 0.4470966 , 0.55483633]],
      dtype=float32)

| Index | Sentence | Similarity (before) | New similarity |
| --- | --- | --- | --- |
| 1 | "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go." | 0.3309 | 0.3309 |
| 2 | "The person box was packed with jelly many dozens of months later." | 0.7219 | 0.7219 |
| 3 | "Standing on one's head at job interviews forms a lasting impression." | 0.1748 | 0.174**7** |
| 4 | "It took him a month to finish the meal." | 0.4471 | 0.447**2** |
| 5 | "He found a leprechaun in his walnut shell." | 0.5548 | 0.554**7** |